In [1]:
# IMPORTS

In [2]:
from dcgan import Generator, Discriminator, weights_init
from train import dcgan_train, get_output
from dataset import WaterBodyGeneratorDataset

import albumentations as A
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML

In [3]:
# CONSTANTS

In [4]:
VEC_SIZE = 100
CHANNELS_OUT = 3
DEVICE = "cuda"
IMAGE_SIZE = (256,256)
BATCH_SIZE = 64
LEARNING_RATE = 2e-4
BETA1 = 0.5
EPOCHS = 3

In [5]:
# MODELS 

In [6]:
generator = Generator(VEC_SIZE, CHANNELS_OUT).to(DEVICE)
generator.apply(weights_init)

Generator(
  (main): ModuleList(
    (0): GeneratorBlock(
      (block): Sequential(
        (0): ConvTranspose2d(100, 8192, kernel_size=(4, 4), stride=(1, 1), bias=False)
        (1): BatchNorm2d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): GeneratorBlock(
      (block): Sequential(
        (0): ConvTranspose2d(8192, 4096, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (2): GeneratorBlock(
      (block): Sequential(
        (0): ConvTranspose2d(4096, 2048, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (3): GeneratorBlock(
      (block): Sequential(
        (0): ConvTranspose2d(2048, 1024, 

In [7]:
discriminator = Discriminator(CHANNELS_OUT).to(DEVICE)
discriminator.apply(weights_init)

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 4.00 GiB total capacity; 2.88 GiB already allocated; 49.22 MiB free; 2.88 GiB reserved in total by PyTorch)

In [ ]:
# DATA

In [ ]:
tf = A.Compose(
    [
        A.Resize(*IMAGE_SIZE),
        A.Normalize(
            mean = [0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0),
        A.pytorch.ToTensorV2()
    ]
)

tf2 = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            mean    = [0.5, 0.5, 0.5],
            std     = [0.5, 0.5, 0.5])
    ]
)

In [ ]:
dataroot = "D:/AI_ML/Kaggle/celeba/"

dataset = dset.ImageFolder(root=dataroot,transform=tf2)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [11]:
dataroot = "D:/AI_ML/Kaggle/Water Bodies Dataset/"
dataset = WaterBodyGeneratorDataset(dataroot, tf2)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [12]:
# TRAINING

In [ ]:
optimizerD = torch.optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
optimizerG = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))

criterion = torch.nn.BCELoss()
img_list = []

In [12]:

FIXED_NOISE = torch.randn(IMAGE_SIZE[0], VEC_SIZE, 1, 1, device=DEVICE)
for epoch in range(EPOCHS):
    dcgan_train(generator=generator, discriminator=discriminator, criterion=criterion, optimizerD=optimizerD, optimizerG=optimizerG, dataloader=dataloader)
    out = get_output(net=generator, input=FIXED_NOISE)
    img_list.append(out)

NameError: name 'discriminator' is not defined

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())